In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers,utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
data=pd.read_csv("data/Car Price Prediction/CarPrice_Assignment.csv")


In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
x=data.drop(["price","car_ID","CarName"],axis=1).copy()
y=data["price"].copy()

In [ ]:
{column: list(x[column].unique()) for column in x.columns if x.dtypes[column]=='object'}


In [ ]:
x["doornumber"]=x["doornumber"].apply(lambda x: 0 if x == 'two' else 1)
numeric_ordering ={'two':2,'three':3,'four':4,'five':5,'six':6,'eight':8,'twelve':12}
x["cylindernumber"] =x["cylindernumber"].replace(numeric_ordering)

In [ ]:
def binary_encode(x, columns, positive_values):
    x =x.copy()
    for column, positive_value in zip(columns, positive_values):
        x[column] = x[column].apply(lambda x: 1 if x == positive_value else 0)
    return x

In [ ]:
binary_features=['fueltype','aspiration','enginelocation',]
binary_positive_values=['diesel','turbo','front']
x=binary_encode(x,columns=binary_features,positive_values=binary_positive_values)

In [ ]:
binary_features = ['fueltype','aspiration','enginelocation']
binary_positive_values=['diesel','turbo','front']
x=binary_encode(x,columns=binary_features,positive_values=binary_positive_values)


In [ ]:
def onehot_encode(x, columns, prefixes):
    x = x.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(x[column], prefix=prefix)
        x=pd.concat([x, dummies], axis=1)
        x=x.drop(column, axis=1)
    return x

In [ ]:
nominal_features=['carbody','drivewheel','enginetype','fuelsystem']
prefixes=['MK','BS','DW','ET','FS']
x=onehot_encode(x,columns=nominal_features,prefixes=prefixes)

In [ ]:
x

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=123)


In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)


In [ ]:
x_train =pd.DataFrame(scaler.transform(x_train),columns=x_train.columns)

In [ ]:
x_test=pd.DataFrame(scaler.transform(x_test),columns=x_test.columns)

In [ ]:
part_x_train,x_val,part_y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=2)

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=42, kernel_initializer='normal', activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])


In [ ]:
history=model.fit(part_x_train,part_y_train,epochs=100,validation_data=(x_val,y_val),batch_size=512)

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
model.predict(x_test).sum()